<a href="https://colab.research.google.com/github/plaban1981/Hugging_Face_transformers_topics/blob/main/Fine_Tuning_MPT_7B_by_Single_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue May 23 05:31:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -qU transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.5 MB/s eta 0:00:00


In [4]:
!pip install -qU einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.1 MB/s eta 0:00:00


In [5]:
import torch
import transformers

model = transformers.AutoModelForCausalLM.from_pretrained(
  'mosaicml/mpt-7b',
  trust_remote_code=True,
  torch_dtype=torch.bfloat16,
)
model.eval()
model.to("cuda:0")

A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b:
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b:
- norm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b:
- flash_attn_triton.py
- norm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b:
- param_init_fns.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b:
- attention.py
- param_init_fns.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


/root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b/14958374ab073ba1030c0caef4ae8380045bae45/attention.py:157: UserWarning: Using `attn_impl: torch`. If your model does not use `alibi` or `prefix_lm` we recommend using `attn_impl: flash` otherwise we recommend using `attn_impl: triton`.
  warnings.warn('Using `attn_impl: torch`. If your model does not use `alibi` or ' + '`prefix_lm` we recommend using `attn_impl: flash` otherwise ' + 'we recommend using `attn_impl: triton`.')


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

MPTForCausalLM(
  (transformer): MPTModel(
    (wte): Embedding(50432, 4096)
    (emb_drop): Dropout(p=0, inplace=False)
    (blocks): ModuleList(
      (0-31): 32 x MPTBlock(
        (norm_1): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (Wqkv): Linear(in_features=4096, out_features=12288, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (norm_2): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (ffn): MPTMLP(
          (up_proj): Linear(in_features=4096, out_features=16384, bias=False)
          (act): GELU(approximate='none')
          (down_proj): Linear(in_features=16384, out_features=4096, bias=False)
        )
        (resid_attn_dropout): Dropout(p=0, inplace=False)
        (resid_ffn_dropout): Dropout(p=0, inplace=False)
      )
    )
    (norm_f): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
)

In [6]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           25Gi       2.4Gi        13Gi        13Mi       9.1Gi        22Gi
Swap:            0B          0B          0B


In [7]:
model_size = sum(t.numel() for t in model.parameters())
print(f"Modelsize: {model_size/1000**2:.1f}M parameters")

Modelsize: 6649.3M parameters


## Tokenizer

In [8]:
tokenizer = transformers.AutoTokenizer.from_pretrained('mosaicml/mpt-7b')

In [9]:
txt = """\
image_path = "/mnt/image.png"

# load image
"""
tokenized_example = tokenizer(txt, return_tensors='pt')

In [10]:
tokenized_example.keys()

dict_keys(['input_ids', 'attention_mask'])

In [11]:
tokenized_example['input_ids']

tensor([[ 5695,    64,  3967,   426, 13357,    78,  2649,    16,  5695,    15,
          8567,     3,   187,   187,     4,  3301,  2460,   187]])

## Generate Output

In [12]:
outputs = model.generate(tokenized_example['input_ids'].to('cuda:0'), 
                         max_new_tokens=150, 
                         do_sample=False, 
                         top_k=5, 
                         top_p=0.95)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


In [13]:
answer = tokenizer.batch_decode(outputs,skip_special_tokens=True)
print(answer)

['image_path = "/mnt/image.png"\n\n# load image\nimage = cv2.imread(image_path)\n\n# convert to grayscale\ngray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)\n\n# blur\nblur = cv2.GaussianBlur(gray, (5, 5), 0)\n\n# find edges\nedged = cv2.Canny(blur, 50, 150)\n\n# find contours\ncontours, hierarchy = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)\n\n# draw contours\nfor cnt in contours:\n    cv2.drawContours']


In [14]:
print(answer[0].rstrip())

image_path = "/mnt/image.png"

# load image
image = cv2.imread(image_path)

# convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# blur
blur = cv2.GaussianBlur(gray, (5, 5), 0)

# find edges
edged = cv2.Canny(blur, 50, 150)

# find contours
contours, hierarchy = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# draw contours
for cnt in contours:
    cv2.drawContours


In [16]:
outputs = model(**tokenized_example.to("cuda:0"))

In [17]:
outputs.logits.shape

torch.Size([1, 18, 50432])

In [18]:
tokenized_example['input_ids'].shape

torch.Size([1, 18])

In [19]:
last_token_output = outputs.logits[0,-1].view(1,-1)
last_token_output.shape

torch.Size([1, 50432])

In [20]:
torch.argmax(last_token_output)

tensor(5695, device='cuda:0')

In [35]:
answer_text = """\
from PIL import Image

image_path = "/mnt/image.png"

# load image
image = cv2.imread(image_path)

# convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# blur
blur = cv2.GaussianBlur(gray, (5, 5), 0)

# find edges
edged = cv2.Canny(blur, 50, 150)

# find contours
contours, hierarchy = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# draw contours
for cnt in contours:
    cv2.drawContours
"""
tokenized_answer = tokenizer.encode(answer_text ,return_tensors='pt')

In [36]:
tokenized_answer

tensor([[ 4064,   367,  3077,  1395, 10882,   187,   187,  5695,    64,  3967,
           426, 13357,    78,  2649,    16,  5695,    15,  8567,     3,   187,
           187,     4,  3301,  2460,   187,  5695,   426, 30105,    19,    15,
           303,  1088,     9,  5695,    64,  3967,    10,   187,   187,     4,
          6455,   281,   650,   698, 25912,   187, 27250,   426, 30105,    19,
            15, 17312,    85,  6573,     9,  5695,    13, 30105,    19,    15,
         38505,    64,    35,  6217,    19,    40, 23544,    10,   187,   187,
             4, 29017,   187, 49857,   426, 30105,    19,    15, 44765,  5622,
           321,     9, 27250,    13,   313,    22,    13,   608,   582,   470,
            10,   187,   187,     4,  1089,  9297,   187,   264,  2400,   426,
         30105,    19,    15,    36, 10059,     9, 49857,    13,  2456,    13,
          7783,    10,   187,   187,     4,  1089, 33183,   187,  1987,  2108,
            13, 19868,   426, 30105,    19,    15,  

In [37]:
tokenized_answer.shape
labels = tokenized_answer[0][0].view(1)
labels

tensor([4064])

In [38]:
for name, param in model.named_parameters():
    print(f"{name}   Modelsize: {param.numel()/1000**2:.1f}M parameters")
    if "31" not in name:
        param.requires_grad = False
    print(name, param.requires_grad)

transformer.wte.weight   Modelsize: 206.6M parameters
transformer.wte.weight False
transformer.blocks.0.norm_1.weight   Modelsize: 0.0M parameters
transformer.blocks.0.norm_1.weight False
transformer.blocks.0.attn.Wqkv.weight   Modelsize: 50.3M parameters
transformer.blocks.0.attn.Wqkv.weight False
transformer.blocks.0.attn.out_proj.weight   Modelsize: 16.8M parameters
transformer.blocks.0.attn.out_proj.weight False
transformer.blocks.0.norm_2.weight   Modelsize: 0.0M parameters
transformer.blocks.0.norm_2.weight False
transformer.blocks.0.ffn.up_proj.weight   Modelsize: 67.1M parameters
transformer.blocks.0.ffn.up_proj.weight False
transformer.blocks.0.ffn.down_proj.weight   Modelsize: 67.1M parameters
transformer.blocks.0.ffn.down_proj.weight False
transformer.blocks.1.norm_1.weight   Modelsize: 0.0M parameters
transformer.blocks.1.norm_1.weight False
transformer.blocks.1.attn.Wqkv.weight   Modelsize: 50.3M parameters
transformer.blocks.1.attn.Wqkv.weight False
transformer.blocks.1.a

## Finetune last empty block

In [39]:
params = sum(t.numel() for t in model.transformer.blocks[-1].parameters())
print(f"Modelsize: {params/1000**2:.1f}M parameters")

Modelsize: 201.3M parameters


In [40]:
lossfct = torch.nn.CrossEntropyLoss()
optimizer = transformers.AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [41]:
labels = tokenized_answer[0][0].view(1) # first batch last token
loss = lossfct(last_token_output,labels.to("cuda:0"))

In [42]:
labels

tensor([4064])

In [43]:
loss.item()

3.671875

In [46]:
model.train()
#loss.backward()
optimizer.step()
optimizer.zero_grad(set_to_none=True)

In [47]:
outputs = model.generate(tokenized_example['input_ids'].to('cuda:0'), 
                         max_new_tokens=50, 
                         do_sample=False, 
                         top_k=5, 
                         top_p=0.95)

In [48]:
answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(answer[0].rstrip())

image_path = "/mnt/image.png"

# load image
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageFilter
from PIL import ImageEnhance
from PIL import ImageOps
from PIL import ImageChops


In [49]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           25Gi       3.6Gi        11Gi        22Mi        10Gi        21Gi
Swap:            0B          0B          0B
